In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''


"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
!pip install pymupdf pdfplumber pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


In [ ]:
!pip install pymupdf python-docx pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 61.5 MB/s eta 0:00:00


In [ ]:
import os
import fitz  # PyMuPDF for PDFs
import docx  # python-docx for DOCX files
import pandas as pd

# Define the folder path where resumes are stored
folder_path = "/content/drive/MyDrive/nlp resume"

# List all PDF and DOCX files in the folder
resume_files = [f for f in os.listdir(folder_path) if f.endswith((".pdf", ".docx"))]

# Initialize a list to store extracted data
resume_data = []

# Function to extract text from DOCX files
def extract_text_from_docx(docx_path):
    try:
        doc = docx.Document(docx_path)
        text = "\n".join([para.text for para in doc.paragraphs])
        return text.strip()
    except Exception as e:
        print(f"Error reading DOCX {docx_path}: {e}")
        return ""

# Loop through each resume and extract text
for resume in resume_files:
    file_path = os.path.join(folder_path, resume)
    text = ""

    try:
        if resume.endswith(".pdf"):
            with fitz.open(file_path) as doc:
                for page in doc:
                    text += page.get_text("text") + " "  # Extract text from each page

        elif resume.endswith(".docx"):
            text = extract_text_from_docx(file_path)

        # Append extracted text to the list
        resume_data.append({"Filename": resume, "Text": text.strip()})

    except Exception as e:
        print(f"Error processing {resume}: {e}")

# Convert to DataFrame
df = pd.DataFrame(resume_data)

# Display DataFrame
df.head()

,Filename,Text
0,Aarav_Mehta_Resume.pdf,Aarav Mehta\nData Scientist | AI & Machine Lea...
1,Neha_Sharma_Resume.pdf,Neha Sharma\nData Scientist | Expert in Deep L...
2,Rahul_Verma_Resume.pdf,Rahul Verma\nData Scientist | Business Intelli...
3,Priya_Agarwal_Resume.pdf,Priya Agarwal\nData Scientist | Expert in Time...
4,Siddharth_Menon_Resume.pdf,Siddharth Menon\nData Scientist | Reinforcemen...


In [ ]:
df.shape

(15, 2)

In [ ]:
import re
import spacy
import pandas as pd
# Load NLP Model
nlp = spacy.load("en_core_web_sm")

# Function to Extract Name (First Named Entity Found)
def extract_name(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":  # Extracts first recognized person entity
            return ent.text.strip()
    return None

# Function to Extract Email
def extract_email(text):
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    email = re.findall(email_pattern, text)
    return email[0] if email else None

# Function to Extract Phone Number
def extract_phone(text):
    phone_pattern = r"\b(?:\+91[-\s]?)?\d{10}\b"
    phone = re.findall(phone_pattern, text)
    return phone[0] if phone else None

# Function to Extract Skills Using Named Entity Recognition (NER)
def extract_skills(text):
    doc = nlp(text)
    skills = [ent.text for ent in doc.ents if ent.label_ == "ORG"]  # Adjust based on model output
    return ", ".join(skills) if skills else None

# Apply Extraction Functions to DataFrame
df["Name"] = df["Text"].apply(extract_name)
df["Email"] = df["Text"].apply(extract_email)
df["Phone"] = df["Text"].apply(extract_phone)
df["Skills"] = df["Text"].apply(extract_skills)

# Display Extracted Data
df[["Filename", "Name", "Email", "Phone", "Skills"]].head()


,Filename,Name,Email,Phone,Skills
0,Aarav_Mehta_Resume.pdf,Aarav Mehta,aarav.mehta@email.com,9876543210,"Data Scientist, AI & Machine Learning Enthusia..."
1,Neha_Sharma_Resume.pdf,Neha Sharma,neha.sharma@email.com,9988776655,"Data Scientist, Deep Learning & NLP, Data Scie..."
2,Rahul_Verma_Resume.pdf,Rahul Verma\nData Scientist,rahul.verma@email.com,9871234567,"Business Intelligence & Big Data, Experienced ..."
3,Priya_Agarwal_Resume.pdf,Proficien\nSkills,priya.agarwal@email.com,9876541234,"Data Scientist, Time Series & Forecasting, Dat..."
4,Siddharth_Menon_Resume.pdf,Siddharth Menon,siddharth.menon@email.com,9898123456,"Data Scientist, Reinforcement Learning & AI Sp..."


In [ ]:
df["Job"]= "DS"
df["Job"].iloc[10:15]="TE"

<ipython-input-7-f3c76bffaa46>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Job"].iloc[10:15]="TE"


In [ ]:
df

,Filename,Text,Name,Email,Phone,Skills,Job
0,Aarav_Mehta_Resume.pdf,Aarav Mehta\nData Scientist | AI & Machine Lea...,Aarav Mehta,aarav.mehta@email.com,9876543210,"Data Scientist, AI & Machine Learning Enthusia...",DS
1,Neha_Sharma_Resume.pdf,Neha Sharma\nData Scientist | Expert in Deep L...,Neha Sharma,neha.sharma@email.com,9988776655,"Data Scientist, Deep Learning & NLP, Data Scie...",DS
2,Rahul_Verma_Resume.pdf,Rahul Verma\nData Scientist | Business Intelli...,Rahul Verma\nData Scientist,rahul.verma@email.com,9871234567,"Business Intelligence & Big Data, Experienced ...",DS
3,Priya_Agarwal_Resume.pdf,Priya Agarwal\nData Scientist | Expert in Time...,Proficien\nSkills,priya.agarwal@email.com,9876541234,"Data Scientist, Time Series & Forecasting, Dat...",DS
4,Siddharth_Menon_Resume.pdf,Siddharth Menon\nData Scientist | Reinforcemen...,Siddharth Menon,siddharth.menon@email.com,9898123456,"Data Scientist, Reinforcement Learning & AI Sp...",DS
5,Ananya_Rao_Resume.pdf,Ananya Rao\nData Scientist | Cloud & MLOps Spe...,Ananya Rao,ananya.rao@email.com,9765432109,"Data Scientist, Cloud & MLOps Specialist, Data...",DS
6,Vikram_Singh_Resume.pdf,Vikram Singh\nData Scientist | Financial Analy...,Vikram Singh,vikram.singh@email.com,9988223344,"Data Scientist, Financial Analytics & Risk Mod...",DS
7,Kiran_Nair_Resume.pdf,Kiran Nair\nData Scientist | Computer Vision &...,Kiran Nair,kiran.nair@email.com,9876612345,"Data Scientist, Computer Vision & Image Proces...",DS
8,Sanya_Kapoor_Resume.pdf,Sanya Kapoor\nData Scientist | Marketing Analy...,Sanya Kapoor,sanya.kapoor@email.com,9887766554,"Data Scientist, Marketing Analytics & Customer...",DS
9,Rajesh_Kumar_Resume.pdf,Rajesh Kumar\nData Scientist | Supply Chain & ...,Rajesh Kumar,rajesh.kumar@email.com,9998887776,"Data Scientist, Supply Chain & Logistics Optim...",DS


In [ ]:
df_profile=df[["Name","Email","Phone","Skills","Job"]]

In [ ]:
df = df_profile[["Skills", "Job"]]


In [ ]:
df

,Skills,Job
0,"Data Scientist, AI & Machine Learning Enthusia...",DS
1,"Data Scientist, Deep Learning & NLP, Data Scie...",DS
2,"Business Intelligence & Big Data, Experienced ...",DS
3,"Data Scientist, Time Series & Forecasting, Dat...",DS
4,"Data Scientist, Reinforcement Learning & AI Sp...",DS
5,"Data Scientist, Cloud & MLOps Specialist, Data...",DS
6,"Data Scientist, Financial Analytics & Risk Mod...",DS
7,"Data Scientist, Computer Vision & Image Proces...",DS
8,"Data Scientist, Marketing Analytics & Customer...",DS
9,"Data Scientist, Supply Chain & Logistics Optim...",DS


In [ ]:
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:


ps = PorterStemmer()

corpus = []
for i in range(len(df)):
    text = df['Skills'][i]  # Extract the text value

    if pd.isna(text):  # Check if the value is missing
        text = ""  # Assign an empty string if missing

    review = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]  # Stemming & stopwords removal
    review = ' '.join(review)

    corpus.append(review)




In [ ]:
corpus

['data scientist ai machin learn enthusiast skill sql ml data scientist abc tech chennai nlp data scienc b tech comput scienc anna univers',
 'data scientist deep learn nlp data scientist nlp tensorflow nltk bert aw gcp data scientist abc ai solut machin learn engin ml sagemak artifici intellig b e inform technolog delhi univers',
 'busi intellig big data experienc data scientist hadoop sa databas manag sql xyz corp data analyst abc solut sql data analyt b tech',
 'data scientist time seri forecast data scientist arima sql financi analyt ltd data scientist appli statist b tech mathemat comput delhi univers',
 'data scientist reinforc learn ai specialist data scientist ppo pytorch robot ai autom ai research scientist ai lab machin learn engin tech innov artifici intellig b e electron anna univers',
 'data scientist cloud mlop specialist data scientist mlop ai ci aw gcp mlop engin cloud ai solut ml aw ai kubernet data scientist chennai iit kharagpur b tech inform technolog vit',
 'data s

W2V CountVectorizer

BAG OF WORDS


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

x=cv.fit_transform(corpus).toarray()
y=pd.get_dummies(df['Job'])
y=y.iloc[:,1].values
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

**word2vec**-google-news-300

In [ ]:
import gensim.downloader
pretrained=gensim.downloader.load('word2vec-google-news-300')

In [ ]:
result=pretrained.most_similar(['india'])
result

[('indian', 0.6967039704322815),
 ('usa', 0.6836211085319519),
 ('pakistan', 0.681516706943512),
 ('chennai', 0.6675503253936768),
 ('america', 0.6589399576187134),
 ('sri_lanka', 0.64982008934021),
 ('canada', 0.6490967869758606),
 ('australia', 0.6368584036827087),
 ('mexico', 0.6239137649536133),
 ('uk', 0.6221641898155212)]

In [ ]:
 !pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 4.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=a16596a359ef059d28a114c13

In [ ]:
 !pip install torch

USING NAIVE BAYES METHOD

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
model=classifier.fit(x_train,y_train)
model.score(x_test,y_test)

1.0

array([False, False, False])

USING RANDOMFOREST METHOD

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
model=classifier.fit(x_train,y_train)
model.score(x_test,y_test)


1.0